## Error study of the 2 bit quantized dino

In [388]:
import pandas as pd

from util.build_dino import build_dino_model
from gptq.modelutils import *

In [389]:
root = "./"

model = build_dino_model(root, backbone="swin-L")
layers = find_layers(model)

use_checkpoint!!!!!!!!!!!!!!!!!!!!!!!!


In [390]:
def get_name(name: str):
    s = ""
    if name.startswith("0."):
        s = f"Backbone_{name[4:]}"
    elif name.startswith("1."):
        s = f"Input_projection_{name[2:]}"
    elif name.startswith("14"):
        s = "Label_classifier"
    elif name.startswith("15"):
        s = f"Bbox_predictor_{name[3:]}"
    elif name.startswith("16"):
        s = "Query_selction_enc_output"
    elif name.startswith("17"):
        s = "Query_selction_enc_out_class_embed"
    elif name.startswith("18"):
        s = f"Query_selction_enc_out_bbox_embed_{name[-8:]}"
    elif int(name.split(".", 1)[0]) >= 8:
        s = f"Decoder_{int(name.split('.', 1)[0]) - 8}_{name.split('.', 1)[1]}"
    else:
        s = f"Encoder_{int(name[0]) - 2}_{name[2:]}"
    return s

In [411]:
# Finding layer number of parameters
layers = model.transformer.encoder.layers + model.transformer.decoder.layers
layers.insert(0, model.input_proj)
layers.insert(0, model.backbone)
layers = layers.append(model.class_embed[0])
layers = layers.append(model.bbox_embed[0])
layers = layers.append(model.transformer.enc_output)
layers = layers.append(model.transformer.enc_out_class_embed)
layers = layers.append(model.transformer.enc_out_bbox_embed)
layers = find_layers(layers)

layers = {get_name(k):v for k,v in layers.items()}
number_of_parameters = {k:int(v.weight.numel()) for k,v in layers.items()}

In [412]:
def get_part(s):
  if s.startswith("Backbone") or  s.startswith("Input_projection"):
    return "backbone"
  elif s.startswith("Encoder") or s.startswith("Decoder"):
    return "transformer"
  elif s.startswith("Query_selction"):
    return "query_selection"
  elif s.startswith("Bbox_predictor") or s.startswith("Label_classifier"):
    return "output_head"

def get_sub_part(s):
  if s.startswith("Backbone"):
    return "backbone"
  elif s.startswith("Input_projection"):
    return "input_projection"
  elif s.startswith("Encoder"):
    return "encoder"
  elif s.startswith("Decoder"):
    return "decoder"
  elif s.startswith("Query_selction"):
    return "query_selection"
  elif s.startswith("Bbox_predictor"):
    return "bbox_predictor"
  elif s.startswith("Label_classifier"):
    return "label_classifier"

In [413]:
dino_swin_ldlq = pd.read_csv('errors/dino-swin-L_ldlq_IP_transformer_backbone_output_head_query_selection_100samples_2bits.csv', names=["layer", "error_ldlq"], header=0)
dino_swin_gptq = pd.read_csv('errors/dino-swin-L_gptq_transformer_backbone_output_head_query_selection_100samples_2bits.csv', names=["layer", "error_gptq"], header=0)
dino_swin = pd.merge(dino_swin_gptq, dino_swin_ldlq, on="layer")
dino_swin["part"] = dino_swin.layer.apply(get_part)
dino_swin["sub_part"] = dino_swin.layer.apply(get_sub_part)
dino_swin["number_of_parameters"] = dino_swin.layer.apply(number_of_parameters.get)
dino_swin["normalized_error_gptq"] = dino_swin.error_gptq / dino_swin.number_of_parameters
dino_swin["normalized_error_ldlq"] = dino_swin.error_ldlq / dino_swin.number_of_parameters
dino_swin["error_gptq"] = dino_swin.error_gptq.round(2)
dino_swin["error_ldlq"] = dino_swin.error_ldlq.round(2)
dino_swin = dino_swin[["layer", "part", "sub_part", "number_of_parameters", "error_gptq", "error_ldlq", "normalized_error_gptq", "normalized_error_ldlq"]]
dino_swin

,layer,part,sub_part,number_of_parameters,error_gptq,error_ldlq,normalized_error_gptq,normalized_error_ldlq
0,Backbone_patch_embed.proj,backbone,backbone,9216,40777.55,76.48,4.424648,0.008299
1,Backbone_layers.0.blocks.0.attn.qkv,backbone,backbone,110592,242.70,185.70,0.002195,0.001679
2,Backbone_layers.0.blocks.0.attn.proj,backbone,backbone,36864,105.53,134.97,0.002863,0.003661
3,Backbone_layers.0.blocks.0.mlp.fc1,backbone,backbone,147456,846797.75,155.21,5.742715,0.001053
4,Backbone_layers.0.blocks.0.mlp.fc2,backbone,backbone,147456,267143.69,328.19,1.811684,0.002226
...,...,...,...,...,...,...,...,...
204,Query_selction_enc_output,query_selection,query_selection,65536,74335.26,63.56,1.134266,0.000970
205,Query_selction_enc_out_class_embed,query_selection,query_selection,23296,32592.27,767.29,1.399050,0.032937
206,Query_selction_enc_out_bbox_embed_layers.0,query_selection,query_selection,65536,50807.65,229.69,0.775263,0.003505
207,Query_selction_enc_out_bbox_embed_layers.1,query_selection,query_selection,65536,41114.84,602.21,0.627363,0.009189


In [423]:
mean_error = dino_swin[["part", "error_gptq", "error_ldlq", "normalized_error_gptq", "normalized_error_ldlq", "number_of_parameters"]].groupby("part").agg({"error_gptq": "mean", "error_ldlq": "mean", "normalized_error_gptq": "mean", "normalized_error_ldlq": "mean", "number_of_parameters": ["mean", "sum"]})
mean_error["error_gptq"] = mean_error["error_gptq"].round(2)
mean_error["error_ldlq"] = mean_error["error_ldlq"].round(2)
mean_error["number_of_parameters"] = mean_error["number_of_parameters"].astype(int)
mean_error["gptq_AP"] = [21.6, 54.4, 52.4, 53.0]
mean_error["ldlq_AP"] = [48.4, 39.3, 57.1, 51.2]
mean_error[[("error_gptq", "mean"), ("error_ldlq", "mean"), ("normalized_error_gptq", "mean"), ("normalized_error_ldlq", "mean"), ("gptq_AP", ""), ("ldlq_AP", ""), ("number_of_parameters", "mean"), ("number_of_parameters", "sum")]].loc[["backbone", "transformer", "query_selection", "output_head"]]

error_gptq error_ldlq normalized_error_gptq  \
                       mean       mean                  mean   
part                                                           
backbone         4656385.05    1703.64              2.226951   
transformer       176637.13     583.21              1.084220   
query_selection    42433.59     351.13              3.388345   
output_head         2369.04     211.56              0.061067   

                normalized_error_ldlq gptq_AP ldlq_AP number_of_parameters  \
                                 mean                                 mean   
part                                                                         
backbone                     0.001294    21.6    48.4              1912290   
transformer                  0.005765    53.0    51.2               176128   
query_selection              0.027469    52.4    57.1                44185   
output_head                  0.023208    54.4    39.3                38848   

                            
                       sum  
part                        
backbone         198878208  
transformer       16908288  
query_selection     220928  
output_head         155392

### Highest error layers


In [352]:
dino_swin.sort_values(by="normalized_error_gptq", ascending=False).head(10)

,layer,part,sub_part,number_of_parameters,error_gptq,error_ldlq,normalized_error_gptq,normalized_error_ldlq
89,Backbone_layers.2.blocks.17.mlp.fc1,backbone,backbone,2359296,1.129683e+08,3656.98,47.882222,0.001550
85,Backbone_layers.2.blocks.16.mlp.fc1,backbone,backbone,2359296,1.082036e+08,3023.03,45.862664,0.001281
81,Backbone_layers.2.blocks.15.mlp.fc1,backbone,backbone,2359296,8.817858e+07,2588.64,37.374953,0.001097
77,Backbone_layers.2.blocks.14.mlp.fc1,backbone,backbone,2359296,5.778317e+07,2189.38,24.491699,0.000928
7,Backbone_layers.0.blocks.1.mlp.fc1,backbone,backbone,147456,2.205139e+06,215.56,14.954558,0.001462
73,Backbone_layers.2.blocks.13.mlp.fc1,backbone,backbone,2359296,3.406924e+07,1848.09,14.440426,0.000783
208,Query_selction_enc_out_bbox_embed_layers.2,query_selection,query_selection,1024,1.331792e+04,92.92,13.005785,0.090743
106,Encoder_0_self_attn.value_proj,transformer,encoder,65536,4.022420e+05,270.40,6.137725,0.004126
69,Backbone_layers.2.blocks.12.mlp.fc1,backbone,backbone,2359296,1.434951e+07,1323.74,6.082115,0.000561
3,Backbone_layers.0.blocks.0.mlp.fc1,backbone,backbone,147456,8.467978e+05,155.21,5.742715,0.001053


In [353]:
dino_swin.sort_values(by="normalized_error_ldlq", ascending=False).head(10)

,layer,part,sub_part,number_of_parameters,error_gptq,error_ldlq,normalized_error_gptq,normalized_error_ldlq
208,Query_selction_enc_out_bbox_embed_layers.2,query_selection,query_selection,1024,13317.92,92.92,13.005785,0.090743
203,Bbox_predictor_layers.2,output_head,bbox_predictor,1024,42.34,75.18,0.041350,0.073417
141,Decoder_0_cross_attn.attention_weights,transformer,decoder,32768,672.99,1416.74,0.020538,0.043236
194,Decoder_5_self_attn.k_proj,transformer,decoder,65536,8985.45,2265.23,0.137107,0.034565
205,Query_selction_enc_out_class_embed,query_selection,query_selection,23296,32592.27,767.29,1.399050,0.032937
196,Decoder_5_self_attn.q_proj,transformer,decoder,65536,14565.74,1225.72,0.222256,0.018703
151,Decoder_1_cross_attn.attention_weights,transformer,decoder,32768,2682.10,554.68,0.081851,0.016928
190,Decoder_5_cross_attn.sampling_offsets,transformer,decoder,65536,6699.32,1022.07,0.102224,0.015596
104,Encoder_0_self_attn.sampling_offsets,transformer,encoder,65536,141620.66,1009.97,2.160960,0.015411
117,Encoder_2_self_attn.attention_weights,transformer,encoder,32768,69700.28,487.87,2.127084,0.014888


### Mean error by model's parts

In [354]:
mean_error = dino_swin[["part", "error_gptq", "error_ldlq", "normalized_error_gptq", "normalized_error_ldlq", "number_of_parameters"]].groupby("part").agg({"error_gptq": "mean", "error_ldlq": "mean", "normalized_error_gptq": "mean", "normalized_error_ldlq": "mean", "number_of_parameters": ["mean", "sum"]})
mean_error["error_gptq"] = mean_error["error_gptq"].round(2)
mean_error["error_ldlq"] = mean_error["error_ldlq"].round(2)
mean_error["number_of_parameters"] = mean_error["number_of_parameters"].astype(int)
mean_error["gptq_AP"] = [21.6, 54.4, 52.4, 53.0]
mean_error["ldlq_AP"] = [48.4, 39.3, 57.1, 51.2]
mean_error[[("error_gptq", "mean"), ("error_ldlq", "mean"), ("normalized_error_gptq", "mean"), ("normalized_error_ldlq", "mean"), ("gptq_AP", ""), ("ldlq_AP", ""), ("number_of_parameters", "mean"), ("number_of_parameters", "sum")]].loc[["backbone", "transformer", "query_selection", "output_head"]]

error_gptq error_ldlq normalized_error_gptq  \
                       mean       mean                  mean   
part                                                           
backbone         4656385.05    1703.64              2.226951   
transformer       176637.13     583.21              1.084220   
query_selection    42433.59     351.13              3.388345   
output_head         2369.04     211.56              0.061067   

                normalized_error_ldlq gptq_AP ldlq_AP number_of_parameters  \
                                 mean                                 mean   
part                                                                         
backbone                     0.001294    21.6    48.4              1912290   
transformer                  0.005765    53.0    51.2               176128   
query_selection              0.027469    52.4    57.1                44185   
output_head                  0.023208    54.4    39.3                38848   

                            
                       sum  
part                        
backbone         198878208  
transformer       16908288  
query_selection     220928  
output_head         155392

In [355]:
mean_error = dino_swin[["sub_part", "error_gptq", "error_ldlq", "normalized_error_gptq", "normalized_error_ldlq", "number_of_parameters"]].groupby("sub_part").agg({"error_gptq": "mean", "error_ldlq": "mean", "normalized_error_gptq": "mean", "normalized_error_ldlq": "mean", "number_of_parameters": ["mean", "sum"]})
mean_error["error_gptq"] = mean_error["error_gptq"].round(2)
mean_error["error_ldlq"] = mean_error["error_ldlq"].round(2)
mean_error["number_of_parameters"] = mean_error["number_of_parameters"].astype(int)
mean_error.loc[["backbone", "input_projection", "encoder", "decoder", "query_selection", "bbox_predictor", "label_classifier"]]

error_gptq error_ldlq normalized_error_gptq  \
                        mean       mean                  mean   
sub_part                                                        
backbone          4832405.39    1687.71              2.262509   
input_projection   255876.55    2102.04              1.338024   
encoder            451572.35     631.89              2.586904   
decoder             11676.00     554.00              0.182610   
query_selection     42433.59     351.13              3.388345   
bbox_predictor       2448.25     189.92              0.050925   
label_classifier     2131.40     276.49              0.091492   

                 normalized_error_ldlq number_of_parameters             
                                  mean                 mean        sum  
sub_part                                                                
backbone                      0.001307              1946511  194651136  
input_projection              0.000972              1056768    4227072  
encoder                       0.005641               212992    7667712  
decoder                       0.005839               154009    9240576  
query_selection               0.027469                44185     220928  
bbox_predictor                0.026988                44032     132096  
label_classifier              0.011869                23296      23296

## Transformer error

In [196]:
trans = dino_swin[dino_swin.part == "transformer"].copy()
trans["trans_part"] = trans.layer.str.split('_', n=2).str[-1]
trans["nb_layer"] = trans.layer.str.split('_', n=3).str[1].astype(int)

In [198]:
trans[["sub_part", "nb_layer", "error_gptq", "error_ldlq", "normalized_error_gptq", "normalized_error_ldlq"]].groupby(["sub_part", "nb_layer"]).mean().loc[["encoder", "decoder"]]

error_gptq  error_ldlq  normalized_error_gptq  \
sub_part nb_layer                                                     
encoder  0         599228.280000  719.701667               3.062971   
         1         540224.816667  641.108333               2.877755   
         2         438957.775000  667.448333               2.475449   
         3         437630.778333  601.911667               2.219882   
         4         414707.791667  613.710000               2.783273   
         5         278684.673333  547.483333               2.102096   
decoder  0          13176.294000  737.988000               0.201975   
         1          11499.625000  427.407000               0.179450   
         2          11667.025000  319.919000               0.183686   
         3          11852.527000  405.243000               0.186188   
         4          10624.167000  511.153000               0.167217   
         5          11236.361000  922.261000               0.177143   

                   normalized_error_ldlq  
sub_part nb_layer                         
encoder  0                      0.006407  
         1                      0.005941  
         2                      0.005994  
         3                      0.004731  
         4                      0.005531  
         5                      0.005242  
decoder  0                      0.009778  
         1                      0.004458  
         2                      0.003265  
         3                      0.003787  
         4                      0.004231  
         5                      0.009515

In [197]:
trans[["sub_part", "trans_part", "error_gptq", "error_ldlq", "normalized_error_gptq", "normalized_error_ldlq"]].groupby(["sub_part", "trans_part"]).mean().sort_index(key=lambda x: x.str[5:]).loc[["encoder", "decoder"]]

error_gptq   error_ldlq  \
sub_part trans_part                                                
encoder  self_attn.attention_weights   7.167297e+04   381.266667   
         self_attn.output_proj         2.026401e+05   218.688333   
         self_attn.sampling_offsets    1.198736e+05   746.206667   
         self_attn.value_proj          2.993676e+05   211.565000   
         linear1                       1.907164e+06   755.916667   
         linear2                       1.087157e+05  1477.720000   
decoder  cross_attn.attention_weights  2.978128e+03   564.711667   
         cross_attn.output_proj        5.137910e+03   241.780000   
         cross_attn.sampling_offsets   4.469888e+03   582.711667   
         cross_attn.value_proj         8.270184e+04    55.356667   
         self_attn.k_proj              7.563838e+03   668.545000   
         self_attn.out_proj            1.646522e+03   265.313333   
         self_attn.q_proj              8.507283e+03   461.458333   
         self_attn.v_proj              3.682638e+03    96.638333   
         linear1                       6.408833e+01   489.408333   
         linear2                       7.866667e+00  2114.028333   

                                       normalized_error_gptq  \
sub_part trans_part                                            
encoder  self_attn.attention_weights                2.187285   
         self_attn.output_proj                      3.092043   
         self_attn.sampling_offsets                 1.829126   
         self_attn.value_proj                       4.567987   
         linear1                                    3.637627   
         linear2                                    0.207359   
decoder  cross_attn.attention_weights               0.090885   
         cross_attn.output_proj                     0.078398   
         cross_attn.sampling_offsets                0.068205   
         cross_attn.value_proj                      1.261930   
         self_attn.k_proj                           0.115415   
         self_attn.out_proj                         0.025124   
         self_attn.q_proj                           0.129811   
         self_attn.v_proj                           0.056193   
         linear1                                    0.000122   
         linear2                                    0.000015   

                                       normalized_error_ldlq  
sub_part trans_part                                           
encoder  self_attn.attention_weights                0.011635  
         self_attn.output_proj                      0.003337  
         self_attn.sampling_offsets                 0.011386  
         self_attn.value_proj                       0.003228  
         linear1                                    0.001442  
         linear2                                    0.002819  
decoder  cross_attn.attention_weights               0.017234  
         cross_attn.output_proj                     0.003689  
         cross_attn.sampling_offsets                0.008891  
         cross_attn.value_proj                      0.000845  
         self_attn.k_proj                           0.010201  
         self_attn.out_proj                         0.004048  
         self_attn.q_proj                           0.007041  
         self_attn.v_proj                           0.001475  
         linear1                                    0.000933  
         linear2                                    0.004032

## Backbone error

In [85]:
backbone = dino_swin[dino_swin.sub_part == "backbone"].copy()
backbone = backbone[1:]
backbone = backbone[~backbone.layer.str.contains("downsample")]
backbone["nb_layer"] = backbone.layer.str.split('.', n=3).str[1].astype(int)
backbone["nb_block"] = backbone.layer.str.split('.', n=4).str[-2].astype(int)

In [86]:
backbone[["nb_layer", "error_gptq", "error_ldlq", "normalized_error_gptq", "normalized_error_ldlq", "number_of_parameters"]].groupby(["nb_layer"]).agg({"error_gptq": "mean", "error_ldlq": "mean", "normalized_error_gptq": "mean", "normalized_error_ldlq": "mean", "number_of_parameters": ["mean", "sum"]})

error_gptq  error_ldlq normalized_error_gptq  \
                  mean        mean                  mean   
nb_layer                                                   
0         4.467895e+05   258.43125              3.031008   
1         3.302421e+05   989.88750              0.561575   
2         6.187644e+06  1745.66000              2.627032   
3         3.843852e+06  3502.03125              0.408933   

         normalized_error_ldlq number_of_parameters             
                          mean                 mean        sum  
nb_layer                                                        
0                     0.002542             110592.0     884736  
1                     0.002104             442368.0    3538944  
2                     0.001049            1769472.0  127401984  
3                     0.000719            7077888.0   56623104

In [87]:
backbone[["nb_layer", "nb_block", "error_gptq", "error_ldlq", "normalized_error_gptq", "normalized_error_ldlq", "number_of_parameters"]].groupby(["nb_layer", "nb_block"]).agg({"error_gptq": "mean", "error_ldlq": "mean", "normalized_error_gptq": "mean", "normalized_error_ldlq": "mean", "number_of_parameters": ["mean", "sum"]})

error_gptq error_ldlq normalized_error_gptq  \
                           mean       mean                  mean   
nb_layer nb_block                                                  
0        0         2.785724e+05   201.0175              1.889864   
         1         6.150066e+05   315.8450              4.172151   
1        0         2.912884e+05  1100.7025              0.495004   
         1         3.691958e+05   879.0725              0.628147   
2        0         3.238435e+04  3868.5375              0.013889   
         1         5.898629e+04  3178.6100              0.025318   
         2         8.049768e+04  2357.5950              0.034791   
         3         1.241036e+05  1879.1175              0.053099   
         4         1.802867e+05  1658.0775              0.077489   
         5         2.443197e+05  1008.1175              0.104824   
         6         3.219970e+05  1403.7600              0.138538   
         7         4.096598e+05  1840.8675              0.175640   
         8         5.084670e+05  2065.6100              0.218800   
         9         6.179589e+05  1372.7725              0.265348   
         10        7.066586e+05  1057.1025              0.304603   
         11        8.480975e+05  1109.6575              0.364780   
         12        3.921798e+06  1218.0875              1.669312   
         13        8.951393e+06  1196.7225              3.800812   
         14        1.497711e+07  1494.8950              6.357551   
         15        2.263464e+07  1243.5125              9.603120   
         16        2.775601e+07  1472.0650             11.774852   
         17        2.900322e+07  1996.7725             12.303802   
3        0         3.623058e+06  4213.0825              0.385230   
         1         4.064646e+06  2790.9800              0.432636   

                  normalized_error_ldlq number_of_parameters            
                                   mean                 mean       sum  
nb_layer nb_block                                                       
0        0                     0.002155             110592.0    442368  
         1                     0.002930             110592.0    442368  
1        0                     0.002281             442368.0   1769472  
         1                     0.001928             442368.0   1769472  
2        0                     0.002073            1769472.0   7077888  
         1                     0.001683            1769472.0   7077888  
         2                     0.001341            1769472.0   7077888  
         3                     0.001086            1769472.0   7077888  
         4                     0.001057            1769472.0   7077888  
         5                     0.000571            1769472.0   7077888  
         6                     0.000838            1769472.0   7077888  
         7                     0.001024            1769472.0   7077888  
         8                     0.001167            1769472.0   7077888  
         9                     0.000898            1769472.0   7077888  
         10                    0.000717            1769472.0   7077888  
         11                    0.000814            1769472.0   7077888  
         12                    0.000817            1769472.0   7077888  
         13                    0.000725            1769472.0   7077888  
         14                    0.000943            1769472.0   7077888  
         15                    0.000756            1769472.0   7077888  
         16                    0.000901            1769472.0   7077888  
         17                    0.001476            1769472.0   7077888  
3        0                     0.000722            7077888.0  28311552  
         1                     0.000717            7077888.0  28311552

In [88]:
output_head = dino_swin[dino_swin.part == "output_head"].copy()
output_head

,layer,part,sub_part,number_of_parameters,error_gptq,error_ldlq,normalized_error_gptq,normalized_error_ldlq
200,Label_classifier,output_head,label_classifier,23296,2131.40,276.49,0.091492,0.011869
201,Bbox_predictor_layers.0,output_head,bbox_predictor,65536,3397.08,72.96,0.051835,0.001113
202,Bbox_predictor_layers.1,output_head,bbox_predictor,65536,3905.32,421.63,0.059590,0.006434
203,Bbox_predictor_layers.2,output_head,bbox_predictor,1024,42.34,75.18,0.041350,0.073417


### One Shot Quantization Model

In [240]:
def get_part_one_shot(s):
  if s.startswith("backbone") or  s.startswith("input_proj"):
    return "backbone"
  elif s.startswith("transformer.enc_"):
    return "query_selection"
  elif s.startswith("transformer"):
    return "transformer"
  elif s.startswith("bbox_embed") or s.startswith("class_embed"):
    return "output_head"

def get_sub_part_one_shot(s):
  if s.startswith("backbone"):
    return "backbone"
  elif s.startswith("input_proj"):
    return "input_projection"
  elif s.startswith("transformer.encoder.layer"):
    return "encoder"
  elif s.startswith("transformer.decoder.layer"):
    return "decoder"
  elif s.startswith("transformer.enc_"):
    return "query_selection"
  elif s.startswith("bbox_embed"):
    return "bbox_predictor"
  elif s.startswith("class_embed"):
    return "label_classifier"

In [241]:
# Finding layer number of parameters
layers_one_shot = find_layers(model)
number_of_parameters_one_shot = {k:int(v.weight.numel()) for k,v in layers_one_shot.items()}

In [330]:
dino_swin_ldlq_one_shot = pd.read_csv('errors/full_model_dino-swin-L_ldlq_IP_100samples_2bits.csv', names=["layer", "error_ldlq"], header=0)
dino_swin_gptq_one_shot = pd.read_csv('errors/full_model_dino-swin-L_gptq_100samples_2bits.csv', names=["layer", "error_gptq"], header=0)
dino_swin_one_shot = pd.merge(dino_swin_gptq_one_shot, dino_swin_ldlq_one_shot, on="layer")
dino_swin_one_shot = dino_swin_one_shot[~dino_swin_one_shot.layer.str.startswith("transformer.decoder.ref")]
dino_swin_one_shot = dino_swin_one_shot[~dino_swin_one_shot.layer.str.startswith("transformer.decoder.bbox")]
dino_swin_one_shot = dino_swin_one_shot[~dino_swin_one_shot.layer.str.startswith("transformer.decoder.class")]
dino_swin_one_shot["part"] = dino_swin_one_shot.layer.apply(get_part_one_shot)
dino_swin_one_shot["sub_part"] = dino_swin_one_shot.layer.apply(get_sub_part_one_shot)
dino_swin_one_shot["number_of_parameters"] = dino_swin_one_shot.layer.apply(number_of_parameters_one_shot.get)
dino_swin_one_shot["normalized_error_gptq"] = dino_swin_one_shot.error_gptq / dino_swin_one_shot.number_of_parameters
dino_swin_one_shot["normalized_error_ldlq"] = dino_swin_one_shot.error_ldlq / dino_swin_one_shot.number_of_parameters
dino_swin_one_shot["error_gptq"] = dino_swin_one_shot.error_gptq.round(2)
dino_swin_one_shot["error_ldlq"] = dino_swin_one_shot.error_ldlq.round(2)
dino_swin_one_shot = dino_swin_one_shot[["layer", "part", "sub_part", "number_of_parameters", "error_gptq", "error_ldlq", "normalized_error_gptq", "normalized_error_ldlq"]]
dino_swin_one_shot

,layer,part,sub_part,number_of_parameters,error_gptq,error_ldlq,normalized_error_gptq,normalized_error_ldlq
0,transformer.encoder.layers.0.self_attn.samplin...,transformer,encoder,65536,141620.66,987.35,2.160960e+00,0.015066
1,transformer.encoder.layers.0.self_attn.attenti...,transformer,encoder,32768,79437.48,279.01,2.424240e+00,0.008515
2,transformer.encoder.layers.0.self_attn.value_proj,transformer,encoder,65536,402241.97,277.82,6.137725e+00,0.004239
3,transformer.encoder.layers.0.self_attn.output_...,transformer,encoder,65536,148757.59,325.25,2.269861e+00,0.004963
4,transformer.encoder.layers.0.linear1,transformer,encoder,524288,2670631.50,673.43,5.093825e+00,0.001284
...,...,...,...,...,...,...,...,...
210,backbone.0.layers.3.blocks.1.mlp.fc2,backbone,backbone,9437184,1845770.50,2278.65,1.955849e-01,0.000241
211,bbox_embed.0.layers.0,output_head,bbox_predictor,65536,1.01,75.59,1.543198e-05,0.001153
212,bbox_embed.0.layers.1,output_head,bbox_predictor,65536,0.01,394.18,1.893616e-07,0.006015
213,bbox_embed.0.layers.2,output_head,bbox_predictor,1024,0.00,91.49,7.910156e-07,0.089341


In [333]:
dino_swin_one_shot.layer = dino_swin_one_shot.layer.str.replace("backbone.0.", "Backbone_", regex=False)
dino_swin_one_shot.layer = dino_swin_one_shot.layer.str.replace("bbox_embed.0.", "Bbox_predictor_", regex=False)
dino_swin_one_shot.layer = dino_swin_one_shot.layer.str.replace("class_embed.0", "Label_classifier", regex=False)
dino_swin_one_shot.layer = dino_swin_one_shot.layer.str.replace("input_proj.", "Input_projection_", regex=False)
dino_swin_one_shot.layer = dino_swin_one_shot.layer.str.replace("transformer.encoder.layers.", "Encoder_", n=1, regex=False)
dino_swin_one_shot.layer = dino_swin_one_shot.layer.str.replace("transformer.decoder.layers.", "Decoder_", n=1, regex=False)
dino_swin_one_shot.loc[dino_swin_one_shot.part == "transformer", "layer"] = dino_swin_one_shot[dino_swin_one_shot.part == "transformer"].layer.str.replace(".", "_", n=1, regex=False)
dino_swin_one_shot

,layer,part,sub_part,number_of_parameters,error_gptq,error_ldlq,normalized_error_gptq,normalized_error_ldlq
0,Encoder_0_self_attn.sampling_offsets,transformer,encoder,65536,141620.66,987.35,2.160960e+00,0.015066
1,Encoder_0_self_attn.attention_weights,transformer,encoder,32768,79437.48,279.01,2.424240e+00,0.008515
2,Encoder_0_self_attn.value_proj,transformer,encoder,65536,402241.97,277.82,6.137725e+00,0.004239
3,Encoder_0_self_attn.output_proj,transformer,encoder,65536,148757.59,325.25,2.269861e+00,0.004963
4,Encoder_0_linear1,transformer,encoder,524288,2670631.50,673.43,5.093825e+00,0.001284
...,...,...,...,...,...,...,...,...
210,Backbone_layers.3.blocks.1.mlp.fc2,backbone,backbone,9437184,1845770.50,2278.65,1.955849e-01,0.000241
211,Bbox_predictor_layers.0,output_head,bbox_predictor,65536,1.01,75.59,1.543198e-05,0.001153
212,Bbox_predictor_layers.1,output_head,bbox_predictor,65536,0.01,394.18,1.893616e-07,0.006015
213,Bbox_predictor_layers.2,output_head,bbox_predictor,1024,0.00,91.49,7.910156e-07,0.089341


In [336]:
dino_swin_merge = pd.merge(dino_swin, dino_swin_one_shot, on=["layer", "part", "sub_part", "number_of_parameters"], suffixes=["", "_one_shot"])
dino_swin_merge = dino_swin_merge[["layer", "part", "sub_part", "number_of_parameters", "error_gptq", "error_gptq_one_shot", "error_ldlq", "error_ldlq_one_shot", "normalized_error_gptq", "normalized_error_gptq_one_shot", "normalized_error_ldlq", "normalized_error_ldlq_one_shot"]]
dino_swin_merge

,layer,part,sub_part,number_of_parameters,error_gptq,error_gptq_one_shot,error_ldlq,error_ldlq_one_shot,normalized_error_gptq,normalized_error_gptq_one_shot,normalized_error_ldlq,normalized_error_ldlq_one_shot
0,Backbone_patch_embed.proj,backbone,backbone,9216,40777.55,40777.55,76.48,68.18,4.424648,4.424648e+00,0.008299,0.007398
1,Backbone_layers.0.blocks.0.attn.qkv,backbone,backbone,110592,242.70,242.70,185.70,184.35,0.002195,2.194556e-03,0.001679,0.001667
2,Backbone_layers.0.blocks.0.attn.proj,backbone,backbone,36864,105.53,105.53,134.97,130.62,0.002863,2.862585e-03,0.003661,0.003543
3,Backbone_layers.0.blocks.0.mlp.fc1,backbone,backbone,147456,846797.75,846797.75,155.21,154.96,5.742715,5.742715e+00,0.001053,0.001051
4,Backbone_layers.0.blocks.0.mlp.fc2,backbone,backbone,147456,267143.69,267143.69,328.19,327.03,1.811684,1.811684e+00,0.002226,0.002218
...,...,...,...,...,...,...,...,...,...,...,...,...
199,Decoder_5_linear2,transformer,decoder,524288,4.32,4.41,3306.48,3234.38,0.000008,8.404236e-06,0.006307,0.006169
200,Label_classifier,output_head,label_classifier,23296,2131.40,107.57,276.49,340.27,0.091492,4.617525e-03,0.011869,0.014607
201,Bbox_predictor_layers.0,output_head,bbox_predictor,65536,3397.08,1.01,72.96,75.59,0.051835,1.543198e-05,0.001113,0.001153
202,Bbox_predictor_layers.1,output_head,bbox_predictor,65536,3905.32,0.01,421.63,394.18,0.059590,1.893616e-07,0.006434,0.006015


In [298]:
backbone = dino_swin_merge[dino_swin_merge.sub_part == "backbone"].copy()
backbone = backbone[1:]
backbone = backbone[~backbone.layer.str.contains("downsample")]
backbone["nb_layer"] = backbone.layer.str.split('.', n=3).str[1].astype(int)
backbone["nb_block"] = backbone.layer.str.split('.', n=4).str[-2].astype(int)

In [300]:
backbone[["nb_layer", "error_gptq", "error_gptq_one_shot", "error_ldlq", "error_ldlq_one_shot", "normalized_error_gptq", "normalized_error_gptq_one_shot", "normalized_error_ldlq", "normalized_error_ldlq_one_shot"]].groupby(["nb_layer"]).mean()

,error_gptq,error_gptq_one_shot,error_ldlq,error_ldlq_one_shot,normalized_error_gptq,normalized_error_gptq_one_shot,normalized_error_ldlq,normalized_error_ldlq_one_shot
nb_layer,,,,,,,,
0,4.467895e+05,4.467895e+05,258.43125,257.256250,3.031008,3.031008,0.002542,0.002528
1,3.302421e+05,3.302421e+05,989.88750,985.531250,0.561575,0.561575,0.002104,0.002102
2,6.187644e+06,6.187644e+06,1745.66000,1748.634722,2.627032,2.627032,0.001049,0.001051
3,3.843852e+06,3.843852e+06,3502.03125,3502.766250,0.408933,0.408933,0.000719,0.000719


In [305]:
backbone[["nb_layer", "nb_block", "error_gptq", "error_gptq_one_shot", "error_ldlq", "error_ldlq_one_shot", "normalized_error_gptq", "normalized_error_gptq_one_shot", "normalized_error_ldlq", "normalized_error_ldlq_one_shot"]].groupby(["nb_layer", "nb_block"]).mean()

error_gptq  error_gptq_one_shot  error_ldlq  \
nb_layer nb_block                                                  
0        0         2.785724e+05         2.785724e+05    201.0175   
         1         6.150066e+05         6.150066e+05    315.8450   
1        0         2.912884e+05         2.912884e+05   1100.7025   
         1         3.691958e+05         3.691958e+05    879.0725   
2        0         3.238435e+04         3.238435e+04   3868.5375   
         1         5.898629e+04         5.898629e+04   3178.6100   
         2         8.049768e+04         8.049768e+04   2357.5950   
         3         1.241036e+05         1.241036e+05   1879.1175   
         4         1.802867e+05         1.802867e+05   1658.0775   
         5         2.443197e+05         2.443197e+05   1008.1175   
         6         3.219970e+05         3.219970e+05   1403.7600   
         7         4.096598e+05         4.096598e+05   1840.8675   
         8         5.084670e+05         5.084670e+05   2065.6100   
         9         6.179589e+05         6.179589e+05   1372.7725   
         10        7.066586e+05         7.066586e+05   1057.1025   
         11        8.480975e+05         8.480975e+05   1109.6575   
         12        3.921798e+06         3.921798e+06   1218.0875   
         13        8.951393e+06         8.951393e+06   1196.7225   
         14        1.497711e+07         1.497711e+07   1494.8950   
         15        2.263464e+07         2.263464e+07   1243.5125   
         16        2.775601e+07         2.775601e+07   1472.0650   
         17        2.900322e+07         2.900322e+07   1996.7725   
3        0         3.623058e+06         3.623058e+06   4213.0825   
         1         4.064646e+06         4.064646e+06   2790.9800   

                   error_ldlq_one_shot  normalized_error_gptq  \
nb_layer nb_block                                               
0        0                    199.2400               1.889864   
         1                    315.2725               4.172151   
1        0                   1101.1850               0.495004   
         1                    869.8775               0.628147   
2        0                   3897.8275               0.013889   
         1                   3210.6975               0.025318   
         2                   2355.5425               0.034791   
         3                   1873.3150               0.053099   
         4                   1657.2550               0.077489   
         5                   1006.1350               0.104824   
         6                   1402.6325               0.138538   
         7                   1845.2500               0.175640   
         8                   2070.9425               0.218800   
         9                   1365.2250               0.265348   
         10                  1057.9275               0.304603   
         11                  1109.8725               0.364780   
         12                  1222.3825               1.669312   
         13                  1194.8725               3.800812   
         14                  1491.5825               6.357551   
         15                  1243.3675               9.603120   
         16                  1474.0725              11.774852   
         17                  1996.5250              12.303802   
3        0                   4219.9375               0.385230   
         1                   2785.5950               0.432636   

                   normalized_error_gptq_one_shot  normalized_error_ldlq  \
nb_layer nb_block                                                          
0        0                               1.889864               0.002155   
         1                               4.172151               0.002930   
1        0                               0.495004               0.002281   
         1                               0.628147               0.001928   
2        0                               0.013889               0.002073   
         1                   

In [338]:
trans = dino_swin_merge[dino_swin_merge.part == "transformer"].copy()
trans["trans_part"] = trans.layer.str.split('_', n=2).str[-1]
trans["nb_layer"] = trans.layer.str.split('_', n=3).str[1].astype(int)
trans

,layer,part,sub_part,number_of_parameters,error_gptq,error_gptq_one_shot,error_ldlq,error_ldlq_one_shot,normalized_error_gptq,normalized_error_gptq_one_shot,normalized_error_ldlq,normalized_error_ldlq_one_shot,trans_part,nb_layer
104,Encoder_0_self_attn.sampling_offsets,transformer,encoder,65536,141620.66,141620.66,1009.97,987.35,2.160960,2.160960,0.015411,0.015066,self_attn.sampling_offsets,0
105,Encoder_0_self_attn.attention_weights,transformer,encoder,32768,79437.48,79437.48,300.39,279.01,2.424240,2.424240,0.009167,0.008515,self_attn.attention_weights,0
106,Encoder_0_self_attn.value_proj,transformer,encoder,65536,402241.97,402241.97,270.40,277.82,6.137725,6.137725,0.004126,0.004239,self_attn.value_proj,0
107,Encoder_0_self_attn.output_proj,transformer,encoder,65536,148757.59,148757.59,338.27,325.25,2.269861,2.269861,0.005162,0.004963,self_attn.output_proj,0
108,Encoder_0_linear1,transformer,encoder,524288,2670631.50,2670631.50,678.41,673.43,5.093825,5.093825,0.001294,0.001284,linear1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Decoder_5_self_attn.v_proj,transformer,decoder,65536,2927.71,2540.75,27.40,26.07,0.044673,0.038769,0.000418,0.000398,self_attn.v_proj,5
196,Decoder_5_self_attn.q_proj,transformer,decoder,65536,14565.74,13603.20,1225.72,1269.69,0.222256,0.207568,0.018703,0.019374,self_attn.q_proj,5
197,Decoder_5_self_attn.out_proj,transformer,decoder,65536,1646.92,1752.32,238.05,209.69,0.025130,0.026738,0.003632,0.003200,self_attn.out_proj,5
198,Decoder_5_linear1,transformer,decoder,524288,48.92,46.92,467.12,457.56,0.000093,0.000089,0.000891,0.000873,linear1,5


In [342]:
trans[["sub_part", "nb_layer", "error_gptq", "error_gptq_one_shot", "error_ldlq", "error_ldlq_one_shot", "normalized_error_gptq", "normalized_error_gptq_one_shot", "normalized_error_ldlq", "normalized_error_ldlq_one_shot"]].groupby(["sub_part", "nb_layer"]).mean().loc[["encoder", "decoder"]]

error_gptq  error_gptq_one_shot  error_ldlq  \
sub_part nb_layer                                                   
encoder  0         599228.280000        599228.280000  719.701667   
         1         540224.816667        510316.486667  641.108333   
         2         438957.775000        392107.373333  667.448333   
         3         437630.778333        380258.250000  601.911667   
         4         414707.791667        358661.191667  613.710000   
         5         278684.673333        246426.546667  547.483333   
decoder  0          13176.294000         11498.637000  737.988000   
         1          11499.625000         10007.013000  427.407000   
         2          11667.025000         10043.829000  319.919000   
         3          11852.527000         10157.759000  405.243000   
         4          10624.167000          9159.858000  511.153000   
         5          11236.361000          9887.736000  922.261000   

                   error_ldlq_one_shot  normalized_error_gptq  \
sub_part nb_layer                                               
encoder  0                  714.110000               3.062971   
         1                  647.080000               2.877755   
         2                  686.580000               2.475449   
         3                  609.460000               2.219882   
         4                  590.323333               2.783273   
         5                  570.741667               2.102096   
decoder  0                  748.897000               0.201975   
         1                  442.010000               0.179450   
         2                  332.241000               0.183686   
         3                  426.145000               0.186188   
         4                  505.398000               0.167217   
         5                  915.857000               0.177143   

                   normalized_error_gptq_one_shot  normalized_error_ldlq  \
sub_part nb_layer                                                          
encoder  0                               3.062971               0.006407   
         1                               2.686860               0.005941   
         2                               2.168611               0.005994   
         3                               1.882374               0.004731   
         4                               2.348546               0.005531   
         5                               1.800466               0.005242   
decoder  0                               0.176374               0.009778   
         1                               0.156402               0.004458   
         2                               0.158156               0.003265   
         3                               0.159681               0.003787   
         4                               0.144411               0.004231   
         5                               0.156338               0.009515   

                   normalized_error_ldlq_one_shot  
sub_part nb_layer                                  
encoder  0                               0.006232  
         1                               0.006015  
         2                               0.006210  
         3                               0.004850  
         4                               0.005522  
         5                               0.005399  
decoder  0                               0.009685  
         1                               0.004558  
         2                               0.003291  
         3                               0.003772  
         4                               0.004204  
         5                               0.009570

In [341]:
trans[["sub_part", "trans_part", "error_gptq", "error_gptq_one_shot", "error_ldlq", "error_ldlq_one_shot", "normalized_error_gptq", "normalized_error_gptq_one_shot", "normalized_error_ldlq", "normalized_error_ldlq_one_shot"]].groupby(["sub_part", "trans_part"]).mean().sort_index(key=lambda x: x.str[5:]).loc[["encoder", "decoder"]]

error_gptq  error_gptq_one_shot  \
sub_part trans_part                                                        
encoder  self_attn.attention_weights   7.167297e+04         6.224906e+04   
         self_attn.output_proj         2.026401e+05         1.926753e+05   
         self_attn.sampling_offsets    1.198736e+05         1.045355e+05   
         self_attn.value_proj          2.993676e+05         2.589311e+05   
         linear1                       1.907164e+06         1.755922e+06   
         linear2                       1.087157e+05         1.126851e+05   
decoder  cross_attn.attention_weights  2.978128e+03         2.716760e+03   
         cross_attn.output_proj        5.137910e+03         5.186345e+03   
         cross_attn.sampling_offsets   4.469888e+03         4.082293e+03   
         cross_attn.value_proj         8.270184e+04         7.025100e+04   
         self_attn.k_proj              7.563838e+03         6.667278e+03   
         self_attn.out_proj            1.646522e+03         1.643693e+03   
         self_attn.q_proj              8.507283e+03         7.598242e+03   
         self_attn.v_proj              3.682638e+03         3.043135e+03   
         linear1                       6.408833e+01         6.145000e+01   
         linear2                       7.866667e+00         7.858333e+00   

                                        error_ldlq  error_ldlq_one_shot  \
sub_part trans_part                                                       
encoder  self_attn.attention_weights    381.266667           388.288333   
         self_attn.output_proj          218.688333           213.981667   
         self_attn.sampling_offsets     746.206667           758.101667   
         self_attn.value_proj           211.565000           213.966667   
         linear1                        755.916667           744.575000   
         linear2                       1477.720000          1499.381667   
decoder  cross_attn.attention_weights   564.711667           565.701667   
         cross_attn.output_proj         241.780000           244.443333   
         cross_attn.sampling_offsets    582.711667           571.501667   
         cross_attn.value_proj           55.356667            56.888333   
         self_attn.k_proj               668.545000           664.425000   
         self_attn.out_proj             265.313333           260.726667   
         self_attn.q_proj               461.458333           469.336667   
         self_attn.v_proj                96.638333            97.096667   
         linear1                        489.408333           492.086667   
         linear2                       2114.028333          2195.373333   

                                       normalized_error_gptq  \
sub_part trans_part                                            
encoder  self_attn.attention_weights                2.187285   
         self_attn.output_proj                      3.092043   
         self_attn.sampling_offsets                 1.829126   
         self_attn.value_proj                       4.567987   
         linear1                                    3.637627   
         linear2                                    0.207359   
decoder  cross_attn.attention_weights               0.090885   
         cross_attn.output_proj                     0.078398   
         cross_attn.sampling_offsets                0.068205   
         cross_attn.value_proj                      1.261930   
         self_attn.k_proj                           0.115415   
         self_attn.out_proj                         0.025124   
         self_attn.q_proj                           0.129811   
         self_attn.v_proj                           0.056193   
         linear1                                    0.000122   
         linear2                                    0.000015   

                                       normalized_error_gptq_one_shot  \
sub_part trans_part                                                     
encoder  self_attn.att